In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split

from pathlib import Path
import re
import json
from tqdm import tqdm

import tensorflow as tf
from keras.models import Sequential
from keras.layers import (
    LSTM,
    Bidirectional,
    Conv1D,
    Dense,
    Flatten,
    Input,
    MaxPooling1D,
    TimeDistributed,
    Dropout
)
from keras.initializers import GlorotUniform
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

2024-08-20 13:30:47.340251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 13:30:47.443718: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 13:30:47.471938: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-20 13:30:47.657245: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-20 13:30:49.048041: W tensorflow/compiler/tf2

In [2]:
conversion_constant = (2.0*16.0)/8192

In [3]:
folder = Path("TCC_data/paper_trained_hand_feature_downsampled_200_filter_4")

### Training

In [4]:
with open(folder/'processed_labeled_filtered_normalized_data.npy', 'rb') as f:
    x_data = np.load(f)

with open(folder/'processed_labeled_filtered_normalized_result.npy', 'rb') as f:
    y_data = np.load(f)

In [5]:
y_data_categorical = to_categorical(y_data,num_classes=np.unique(y_data).shape[0])

x_train, x_val, y_train, y_val = train_test_split(x_data,y_data_categorical,
                                                    test_size=0.3,
                                                    random_state=42,
                                                    stratify=y_data_categorical)

x_val, _, y_val, _ = train_test_split(x_val,y_val,
                                        test_size=0.34,
                                        random_state=42,
                                        stratify=y_val)

#### Sequential Model

In [6]:
model_name = "vanilla"
# Define the model
model = Sequential()

# Add LSTM layer with 100 units
model.add(LSTM(100, input_shape=(10, 9), return_sequences=False, kernel_initializer=GlorotUniform(), activation='relu'))

# Add Dropout layer
model.add(Dropout(0.2))  # Adjust dropout rate as needed

# Add output layer
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["categorical_accuracy","accuracy"])

# Summary of the model
model.summary()

I0000 00:00:1724171451.767279  523217 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-20 13:30:52.262575: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/kenny/miniforge3/envs/fd/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100)            │        44,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │           606 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,606 (174.24 KB)

 Trainable params: 44,606 (174.24 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
checkpoint_callback = ModelCheckpoint(
    filepath=folder/f'best_{model_name}_model.keras',       # Filepath to save the model
    monitor='val_accuracy',            # Monitor validation accuracy
    save_best_only=True,               # Save only the best model
    save_weights_only=False,           # Save the full model (architecture + weights)
    mode='max',                        # Save when the monitored quantity is maximized
    verbose=1                          # Verbosity mode
)

In [8]:
# Train
history = model.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_data=(x_val,y_val),
    callbacks=[checkpoint_callback],
    shuffle=True
    )

Epoch 1/100
2800/2802 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4057 - categorical_accuracy: 0.4057 - loss: 1.4139
Epoch 1: val_accuracy improved from -inf to 0.48095, saving model to TCC_data/paper_trained_hand_feature_downsampled_200_filter_4/best_vanilla_model.keras
2802/2802 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - accuracy: 0.4057 - categorical_accuracy: 0.4057 - loss: 1.4138 - val_accuracy: 0.4810 - val_categorical_accuracy: 0.4810 - val_loss: 1.2415
Epoch 2/100
2799/2802 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5529 - categorical_accuracy: 0.5529 - loss: 1.0667
Epoch 2: val_accuracy improved from 0.48095 to 0.62727, saving model to TCC_data/paper_trained_hand_feature_downsampled_200_filter_4/best_vanilla_model.keras
2802/2802 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.5530 - categorical_accuracy: 0.5530 - loss: 1.0666 - val_accuracy: 0.6273 - val_categorical_accuracy: 0.6273 - val_loss: 0.8835
Epoch 3/100
2800/2802 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6145 -

In [9]:
with open(folder/f'training_history_{model_name}.json', 'w') as f:
    json.dump(history.history, f)

In [10]:
y_pred = model.predict(x_train)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_train_classes = np.argmax(y_train, axis=1)

# Calculate precision and recall for multi-class classification
precision = precision_score(y_train_classes, y_pred_classes, average='macro')
recall = recall_score(y_train_classes, y_pred_classes, average='macro')

print(f"Precision: {precision}")
print(f"Recall: {recall}")

2802/2802 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
Precision: 0.7508056696920141
Recall: 0.7115604320729743


#### Stacked Model

In [11]:
model_name = "stacked"
# Define the model
model_stacked = Sequential()

# Add LSTM layer with 100 units
model_stacked.add(LSTM(50, input_shape=(10, 9), return_sequences=True, kernel_initializer=GlorotUniform(), activation='relu'))
model_stacked.add(LSTM(50, kernel_initializer=GlorotUniform(), activation='relu'))

# Add Dropout layer
model_stacked.add(Dropout(0.2))  # Adjust dropout rate as needed

# Add output layer
model_stacked.add(Dense(6, activation='softmax'))

# Compile the model_stacked
model_stacked.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["categorical_accuracy","accuracy"])

# Summary of the model_stacked
model_stacked.summary()

/home/kenny/miniforge3/envs/fd/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 10, 50)         │        12,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           306 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,506 (126.98 KB)

 Trainable params: 32,506 (126.98 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
checkpoint_callback = ModelCheckpoint(
    filepath=folder/f'best_{model_name}_model.keras',# Filepath to save the model
    monitor='val_accuracy',            # Monitor validation accuracy
    save_best_only=True,               # Save only the best model
    save_weights_only=False,           # Save the full model (architecture + weights)
    mode='max',                        # Save when the monitored quantity is maximized
    verbose=1                          # Verbosity mode
)

In [13]:
# Train
history2 = model_stacked.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_data=(x_val,y_val),
    callbacks=[checkpoint_callback],
    shuffle=True
)

Epoch 1/100
2802/2802 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3937 - categorical_accuracy: 0.3937 - loss: 1.4264
Epoch 1: val_accuracy improved from -inf to 0.44333, saving model to TCC_data/paper_trained_hand_feature_downsampled_200_filter_4/best_stacked_model.keras
2802/2802 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - accuracy: 0.3937 - categorical_accuracy: 0.3937 - loss: 1.4264 - val_accuracy: 0.4433 - val_categorical_accuracy: 0.4433 - val_loss: 1.3193
Epoch 2/100
2798/2802 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4845 - categorical_accuracy: 0.4845 - loss: 1.2197
Epoch 2: val_accuracy improved from 0.44333 to 0.62281, saving model to TCC_data/paper_trained_hand_feature_downsampled_200_filter_4/best_stacked_model.keras
2802/2802 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.4846 - categorical_accuracy: 0.4846 - loss: 1.2194 - val_accuracy: 0.6228 - val_categorical_accuracy: 0.6228 - val_loss: 0.8850
Epoch 3/100
2800/2802 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6114 -

In [14]:
with open(folder/f'training_history_{model_name}.json', 'w') as f:
    json.dump(history2.history, f)

In [15]:
y_pred = model_stacked.predict(x_train)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_train_classes = np.argmax(y_train, axis=1)

# Calculate precision and recall for multi-class classification
precision = precision_score(y_train_classes, y_pred_classes, average='macro')
recall = recall_score(y_train_classes, y_pred_classes, average='macro')

print(f"Precision: {precision}")
print(f"Recall: {recall}")

2802/2802 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
Precision: 0.7421124785368752
Recall: 0.7025072429341992


#### Bidirectional Model

In [16]:
model_name = "bidirectional"

In [17]:
# Define the model
model_bi = Sequential()

# Add LSTM layer with 100 units
model_bi.add(Bidirectional(LSTM(units=100,kernel_initializer=GlorotUniform(), activation='relu'), input_shape=(10, 9)))

# Add Dropout layer
model_bi.add(Dropout(0.2))  # Adjust dropout rate as needed

# Add output layer
model_bi.add(Dense(6, activation='softmax'))

# Compile the model_bi
model_bi.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["categorical_accuracy","accuracy"])

# Summary of the model_bi
model_bi.summary()

/home/kenny/miniforge3/envs/fd/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 200)            │        88,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │         1,206 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 89,206 (348.46 KB)

 Trainable params: 89,206 (348.46 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
checkpoint_callback = ModelCheckpoint(
    filepath=folder/f'best_{model_name}_model.keras',# Filepath to save the model
    monitor='val_accuracy',            # Monitor validation accuracy
    save_best_only=True,               # Save only the best model
    save_weights_only=False,           # Save the full model (architecture + weights)
    mode='max',                        # Save when the monitored quantity is maximized
    verbose=1                          # Verbosity mode
)

In [19]:
# Train
history3 = model_bi.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_data=(x_val,y_val),
    callbacks=[checkpoint_callback],
    shuffle=True
)

Epoch 1/100
2795/2802 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4181 - categorical_accuracy: 0.4181 - loss: 1.3900
Epoch 1: val_accuracy improved from -inf to 0.57549, saving model to TCC_data/paper_trained_hand_feature_downsampled_200_filter_4/best_bidirectional_model.keras
2802/2802 ━━━━━━━━━━━━━━━━━━━━ 23s 8ms/step - accuracy: 0.4182 - categorical_accuracy: 0.4182 - loss: 1.3898 - val_accuracy: 0.5755 - val_categorical_accuracy: 0.5755 - val_loss: 1.0216
Epoch 2/100
2802/2802 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5810 - categorical_accuracy: 0.5810 - loss: 0.9891
Epoch 2: val_accuracy improved from 0.57549 to 0.61749, saving model to TCC_data/paper_trained_hand_feature_downsampled_200_filter_4/best_bidirectional_model.keras
2802/2802 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.5810 - categorical_accuracy: 0.5810 - loss: 0.9891 - val_accuracy: 0.6175 - val_categorical_accuracy: 0.6175 - val_loss: 0.8839
Epoch 3/100
2797/2802 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accura

In [20]:
with open(folder/f'training_history_{model_name}.json', 'w') as f:
    json.dump(history3.history, f)

In [21]:
y_pred = model_bi.predict(x_train)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_train_classes = np.argmax(y_train, axis=1)

# Calculate precision and recall for multi-class classification
precision = precision_score(y_train_classes, y_pred_classes, average='macro')
recall = recall_score(y_train_classes, y_pred_classes, average='macro')

print(f"Precision: {precision}")
print(f"Recall: {recall}")

2802/2802 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
Precision: 0.7681926655680772
Recall: 0.7446894956387636
